In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from openai import OpenAI
from IPython.display import Markdown,display

In [2]:
load_dotenv(override=True)
openai = OpenAI()

openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    print(f"OpenAI API Key is present and the first few cahracters of the key looks like this {openai_api_key[:8]}")
else:
    print("OpenAI API Key did not load. Please further troublshoot")

OpenAI API Key is present and the first few cahracters of the key looks like this sk-proj-


In [3]:
messages = [{
    'role':'user',
    'content':'Hi ChatGPT! This is my first message to you! Hope you are doing well.'
}]

response = openai.chat.completions.create(model="gpt-4o-mini",messages=messages)

answer = response.choices[0].message.content

print(answer)

Hello! I’m doing well, thank you! It’s great to hear from you. How can I assist you today?


In [4]:
headers={
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    url: str
    title: str
    text: str
    
    def __init__(self,url):
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script","style","img","input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [5]:
ed = Website("https://edwarddonner.com")

print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

In [6]:
system_prompt = "You are an assisant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [7]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
    please provide a short summary of this website in markdown. \
    If it includes news or announcements, then summarize these too.\n\n"

    user_prompt += website.text

    return user_prompt

print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows;     please provide a short summary of this website in markdown.     If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untap

In [8]:
messages = [
    {'role':'system', 'content':'You are a snarky assistant'},
    {'role':'user','content':'What is 2+2?'}
]

In [9]:
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)

answer = response.choices[0].message.content

print(answer)

Oh, you’re really going for the tough questions today, huh? Well, I’ll humor you—2 + 2 equals 4. Shocking, I know!


In [10]:
def messages_for(website):
    return[
        {'role':'system', 'content': system_prompt},
        {'role':'user', 'content':user_prompt_for(website)}
    ]

In [11]:
messages_for(ed)

[{'role': 'system',
  'content': 'You are an assisant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows;     please provide a short summary of this website in markdown.     If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can

In [12]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(model="gpt-4o-mini",
                                             messages=messages_for(website))
    return response.choices[0].message.content

In [13]:
summarize("https://edwarddonner.com")

'# Summary of Edward Donner\'s Website\n\nEdward Donner\'s website serves as a personal platform where he shares his interests in coding, large language models (LLMs), and electronic music. As the co-founder and CTO of Nebula.io, he is focused on leveraging AI to help people discover their potential, particularly in the talent management space. Previously, he founded the AI startup untapt, which was acquired in 2021.\n\n## News and Announcements\n\n- **May 28, 2025**: Announcement of courses aimed at developing expertise and leadership in LLMs.\n- **May 18, 2025**: Information about the 2025 AI Executive Briefing.\n- **April 21, 2025**: Launch of "The Complete Agentic AI Engineering Course."\n- **January 23, 2025**: Resources shared for the "LLM Workshop – Hands-on with Agents."'

In [14]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

display_summary("https://edwarddonner.com")

# Summary of the Edward Donner Website

The website belongs to Ed Donner, a programmer and enthusiast in the field of Large Language Models (LLMs). Ed is the co-founder and CTO of Nebula.io, which focuses on employing AI to help individuals discover their potential and enhance talent management for recruiters. Previously, he founded the AI startup Untapt, which was acquired in 2021. 

## Key Features
- **Personal Interests**: Ed enjoys coding, experimenting with LLMs, DJing, and producing electronic music. He also engages with the tech community through platforms like Hacker News.
- **Current Projects**: Highlights include innovative LLM applications tailored for talent management and a patented matching model.

## News & Announcements
- **Courses and Workshops**:
  - **Connecting my courses – become an LLM expert and leader**: May 28, 2025
  - **2025 AI Executive Briefing**: May 18, 2025
  - **The Complete Agentic AI Engineering Course**: April 21, 2025
  - **LLM Workshop – Hands-on with Agents – resources**: January 23, 2025

The website encourages connection and collaboration within the LLM community.

In [15]:
display_summary("https://cnn.com")

# CNN Website Summary

CNN provides comprehensive coverage of breaking news from the U.S. and around the world. The platform features a variety of topics including politics, business, health, entertainment, sports, and science, along with live updates on significant global events.

## Key News Highlights
- **Trump's Legislative Agenda**: Republicans are racing to meet a deadline for Trump's new domestic policy bill, which has progressed in the Senate.
- **Iran's Nuclear Program**: The UN's nuclear watchdog reports Iran could start enriching uranium again soon, as an Israeli strike reportedly targets an Iranian prison.
- **Ukraine Conflict**: The effectiveness of Russia's recent military offensives is under scrutiny as Ukraine faces significant aerial attacks.
- **Jeff Bezos' Wedding**: Details about the lavish wedding of Jeff Bezos and Lauren Sanchez are featured, sparking discussions on cultural and social issues too.
- **LGBTQ+ Rights**: Protests and celebrations are noted around Pride Month in the U.S., alongside reports about legislative actions affecting LGBTQ+ communities.

The site also includes various multimedia content, such as videos on current articles and ongoing global crises, ensuring viewers are up-to-date with the latest developments.

In [16]:
display_summary("https://anthropic.com")

# Anthropic Overview

Anthropic is a company dedicated to building AI technologies with a focus on safety and human benefit. The key features of the website include:

- **Claude**: Anthropic's advanced AI models, including Claude Opus 4 and Claude Sonnet 4, which are designed for various applications including coding and AI agents. 
- **API Access**: Developers are encouraged to build custom applications using the Claude API, with supporting documentation provided.
- **Research**: The site includes resources and research on AI safety, societal impacts, and machine interpretability, emphasizing responsible AI development.
- **Commitments**: Information on Anthropic’s transparency, responsible scaling policy, and security measures.
- **Anthropic Academy**: Educational resources aimed at teaching users how to effectively build with Claude.

## Recent Announcements

- **ISO 42001 Certification**: Anthropic has received certification in ISO 42001, highlighting its commitment to safe AI practices.
- **Release of Claude Opus 4**: The most intelligent AI model to date, showcasing advancements in AI capabilities.

The overarching goal of Anthropic is to develop powerful AI systems that prioritize the long-term well-being of society.